In [2]:
import pandas as pd

# Load the dataset from your computer
# Replace 'your_dataset.csv' with the actual file path of your dataset
df = pd.read_csv(r"C:\Users\VIVEK KUMAR SINGH\Downloads\WSN-DS.csv")
print(df.columns)
# Display the first few rows of the dataset to verify
print(df.head())

# Strip leading and trailing spaces from column names
df.columns = df.columns.str.strip()

# Group by Time and calculate the required metrics
aggregated_df = df.groupby("Time").agg(
    Number_of_Nodes=("id", "count"),
    Number_of_CH=("Is_CH", "sum"),
    Sum_Dist_To_CH=("Dist_To_CH", "sum"),
    Avg_Dist_To_CH=("Dist_To_CH", "mean"),
    Avg_ADV_S=("ADV_S", "mean"),
    Avg_ADV_R=("ADV_R", "mean"),
    Avg_JOIN_S=("JOIN_S", "mean"),
    Avg_JOIN_R=("JOIN_R", "mean"),
    Avg_SCH_S=("SCH_S", "mean"),
    Avg_SCH_R=("SCH_R", "mean"),
    Avg_Data_Sent=("DATA_S", "mean"),
    Sum_Dist_CH_To_BS=("dist_CH_To_BS", "sum"),
    Avg_Dist_CH_To_BS=("dist_CH_To_BS", "mean"),
    Avg_Send_Code=("send_code", "mean"),
    Sum_Expanded_Energy=("Expaned Energy", "sum"),
    Avg_Expanded_Energy=("Expaned Energy", "mean")
).reset_index()

# Display the resulting dataset
print(aggregated_df)


Index([' id', ' Time', ' Is_CH', ' who CH', ' Dist_To_CH', ' ADV_S', ' ADV_R',
       ' JOIN_S', ' JOIN_R', ' SCH_S', ' SCH_R', 'Rank', ' DATA_S', ' DATA_R',
       ' Data_Sent_To_BS', ' dist_CH_To_BS', ' send_code ', 'Expaned Energy',
       'Attack type'],
      dtype='object')
       id   Time   Is_CH   who CH   Dist_To_CH   ADV_S   ADV_R   JOIN_S  \
0  101000     50       1   101000      0.00000       1       0        0   
1  101001     50       0   101044     75.32345       0       4        1   
2  101002     50       0   101010     46.95453       0       4        1   
3  101003     50       0   101044     64.85231       0       4        1   
4  101004     50       0   101010      4.83341       0       4        1   

    JOIN_R   SCH_S   SCH_R  Rank   DATA_S   DATA_R   Data_Sent_To_BS  \
0       25       1       0     0        0     1200                48   
1        0       0       1     2       38        0                 0   
2        0       0       1    19       41        0  

In [17]:
!pip install --upgrade scikit-learn imbalanced-learn


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/a1/a6/c5b78606743a1f28eae8f11973de6613a5ee87366796583fb74c67d54939/scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for imbalanced-learn from https://files.pythonhosted.org/packages/9d/41/721fec82606242a2072ee909086ff918dfad7d0199a9dfd4928df9c72494/imbalanced_learn-0.13.0-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for sklearn-compat<1,>=0.1 from https://files.pythonhosted.org/packages/f0/a8/ad69cf130fbd017660cdd64abbef3f28135d9e2e15fe3002e03c5be0ca38/sklearn_compat-0.1.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0

In [1]:
import sklearn
import imblearn

print("scikit-learn version:", sklearn.__version__)
print("imblearn version:", imblearn.__version__)


scikit-learn version: 1.6.1
imblearn version: 0.13.0


In [4]:
import numpy as np
def add_random_noise(df, noise_factor=0.01, num_samples=19000):
    # Create additional synthetic rows
    synthetic_data = []
    numeric_columns = df.select_dtypes(include=[np.number]).columns

    for _ in range(num_samples):
        # Randomly select a row and add random noise
        random_row = df.sample(n=1, random_state=42).copy()
        for col in numeric_columns:
            noise = np.random.normal(0, noise_factor, size=random_row[col].shape)
            random_row[col] += noise
            # Ensure that no values go below 0
            random_row[col] = random_row[col].clip(lower=0)
        
        # Round specific columns to ensure integer values
        if 'number of nodes' in random_row.columns:
            random_row['number of nodes'] = random_row['number of nodes'].round().astype(int)
        if 'number of CH' in random_row.columns:
            random_row['number of CH'] = random_row['number of CH'].round().astype(int)
        
        synthetic_data.append(random_row)

    # Concatenate the original dataframe with the synthetic data
    augmented_df = pd.concat([df] + synthetic_data, ignore_index=True)
    return augmented_df

# Apply the noise injection to generate 19,305 more samples
augmented_df = add_random_noise(aggregated_df, num_samples=19000)

# Display the augmented dataset
print(augmented_df.shape)  # Should now have 195 + 19,000 = 19,195 rows




(19196, 17)


In [5]:
print(augmented_df)

              Time  Number_of_Nodes  Number_of_CH  Sum_Dist_To_CH  \
0        50.000000       300.000000     17.000000     7491.895690   
1        53.000000     13600.000000    672.000000   407950.698010   
2       100.000000       300.000000     13.000000     8437.870040   
3       103.000000     13500.000000    618.000000   434538.738890   
4       150.000000       300.000000     13.000000     8895.005470   
...            ...              ...           ...             ...   
19191  2300.000404         9.995983      0.000000        0.000000   
19192  2299.997388        10.019269      0.000000        0.000000   
19193  2300.005235        10.001293      0.000000        0.010013   
19194  2299.980787        10.020111      0.001894        0.013505   
19195  2300.003327         9.998322      0.003375        0.001910   

       Avg_Dist_To_CH  Avg_ADV_S  Avg_ADV_R  Avg_JOIN_S  Avg_JOIN_R  \
0           24.972986   0.056667   5.330000    0.943333    0.943333   
1           29.996375   0.090

In [6]:
print(augmented_df.columns)

Index(['Time', 'Number_of_Nodes', 'Number_of_CH', 'Sum_Dist_To_CH',
       'Avg_Dist_To_CH', 'Avg_ADV_S', 'Avg_ADV_R', 'Avg_JOIN_S', 'Avg_JOIN_R',
       'Avg_SCH_S', 'Avg_SCH_R', 'Avg_Data_Sent', 'Sum_Dist_CH_To_BS',
       'Avg_Dist_CH_To_BS', 'Avg_Send_Code', 'Sum_Expanded_Energy',
       'Avg_Expanded_Energy'],
      dtype='object')


In [8]:
# Ensure the required columns exist in the DataFrame
# Sum_Expanded_Energy and Sum_Dist_To_CH should be present in augmented_df

# Constants
flying_power = 71.33  # W
flying_speed = 4.7    # m/s
battery_capacity_joules = 144000  # J

# Calculate the minimum UAV for each row
augmented_df['minimum_UAV'] = np.ceil(
    (
        augmented_df['Sum_Expanded_Energy'] +
        (augmented_df['Sum_Dist_To_CH'] * flying_power / flying_speed)
    ) / battery_capacity_joules
).astype(int)


# Display the updated DataFrame
print(augmented_df.head())


    Time  Number_of_Nodes  Number_of_CH  Sum_Dist_To_CH  Avg_Dist_To_CH  \
0   50.0            300.0          17.0      7491.89569       24.972986   
1   53.0          13600.0         672.0    407950.69801       29.996375   
2  100.0            300.0          13.0      8437.87004       28.126233   
3  103.0          13500.0         618.0    434538.73889       32.188055   
4  150.0            300.0          13.0      8895.00547       29.650018   

   Avg_ADV_S  Avg_ADV_R  Avg_JOIN_S  Avg_JOIN_R  Avg_SCH_S  Avg_SCH_R  \
0   0.056667   5.330000    0.943333    0.943333   0.053333   0.943333   
1   0.090515   7.218897    0.950588    0.949926   0.264265   0.946324   
2   0.043333   4.290000    0.956667    0.956667   0.040000   0.956667   
3   0.131778   7.989778    0.946815    0.875407   0.220593   0.896000   
4   0.043333   4.290000    0.956667    0.956667   0.043333   0.956667   

   Avg_Data_Sent  Sum_Dist_CH_To_BS  Avg_Dist_CH_To_BS  Avg_Send_Code  \
0      55.853333         2213.74496  

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Features to use for prediction
features = [
    "Number_of_Nodes", "Number_of_CH", "Avg_Dist_To_CH", 
    "Avg_ADV_S", "Avg_ADV_R", "Avg_JOIN_S", "Avg_JOIN_R", 
    "Avg_SCH_S", "Avg_SCH_R", "Avg_Data_Sent", 
    "Avg_Dist_CH_To_BS", "Avg_Send_Code", "Avg_Expanded_Energy"
]

target = "minimum_UAV"

# Ensure the necessary columns exist in the dataset
assert set(features).issubset(augmented_df.columns), "Some features are missing in the DataFrame."
assert target in augmented_df.columns, "Target column 'minimum_UAV' is missing."

# Split the data into features (X) and target (y)
X = augmented_df[features]
y = augmented_df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features for better neural network performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  # Single output for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, validation_split=0.1, epochs=50, batch_size=32, verbose=1)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {test_mae}")

# Make predictions
y_pred = model.predict(X_test)
y_pred = np.rint(y_pred).astype(int)  # Round to the nearest integer to match 'minimum_UAV'

# Metrics
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, y_pred)}")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred)}")


Epoch 1/50
432/432 [==============================] - 3s 4ms/step - loss: 0.7051 - mae: 0.2134 - val_loss: 0.3022 - val_mae: 0.0705
Epoch 2/50
432/432 [==============================] - 1s 3ms/step - loss: 0.2634 - mae: 0.1106 - val_loss: 0.1623 - val_mae: 0.0391
Epoch 3/50
432/432 [==============================] - 1s 3ms/step - loss: 0.2017 - mae: 0.0857 - val_loss: 0.0295 - val_mae: 0.0231
Epoch 4/50
432/432 [==============================] - 1s 3ms/step - loss: 0.1738 - mae: 0.0670 - val_loss: 0.1349 - val_mae: 0.0307
Epoch 5/50
432/432 [==============================] - 1s 3ms/step - loss: 0.1106 - mae: 0.0530 - val_loss: 0.0788 - val_mae: 0.0540
Epoch 6/50
432/432 [==============================] - 1s 3ms/step - loss: 0.2368 - mae: 0.0541 - val_loss: 0.0385 - val_mae: 0.0254
Epoch 7/50
432/432 [==============================] - 1s 3ms/step - loss: 0.1358 - mae: 0.0422 - val_loss: 0.1827 - val_mae: 0.0291
Epoch 8/50
432/432 [==============================] - 2s 4ms/step - loss: 0.

In [10]:
# Define an acceptable error margin
tolerance = 1

# Calculate the number of accurate predictions
accurate_predictions = np.sum(np.abs(y_test - y_pred.flatten()) <= tolerance)

# Calculate accuracy as a percentage
accuracy = (accurate_predictions / len(y_test)) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 99.87%
